<a href="https://colab.research.google.com/github/GNuSeekK/MachineLearning/blob/main/DB_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 드라이브 Mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# Frame_work , Library import

In [44]:
!pip install pymysql
 
#드라이브 마운트
import pandas as pd
import numpy as np
import random
import os, sys
import pymysql
import datetime as dt
import requests
from bs4 import BeautifulSoup
import multiprocessing as mp
from multiprocessing import Pool, Manager
from dateutil.relativedelta import relativedelta

#Function


## Function **db_connecting_aws**

In [4]:
def db_connecting_aws(db_name):
    path = r'/content/gdrive/My Drive/DB 정보/AWS'
    f = open(path)
    id, pw, host = f.read().split()
    db = pymysql.connect(
        user = id,
        port = 3306,
        passwd = pw,
        host = host,
        db = db_name,
        charset = 'utf8',
        cursorclass = pymysql.cursors.DictCursor
    )
    return db

### 실제 DB에 연결하기
* cpu의 갯수만큼 연결한다.
* 하나로 쓸 db, 여러개로 사용할 db_list를 따로 구성한다

In [5]:
#db 작업
 
db = db_connecting_aws('stockdb')

## Function insert_data

In [ ]:
#데이터 넣기, 입력값 // db = 데이터베이스, df = 데이터프레임
def insert_data(db,df,tbl):
    df_list = df.values.tolist()
    slicing = 5000
    rep = int(len(df_list)/slicing) + 1 #반복횟수
    for num in range(rep):
        if num != (rep-1):
            df_sql = df_list[num*slicing:(num+1)*slicing]
        else:
            df_sql = df_list[num*slicing:]
        sql_list = []
        for value in df_sql:
            tmp = '(' + ', '.join(value) + ')'
            sql_list.append(tmp)
    
        sql = 'insert into %s values ' %tbl + ', '.join(sql_list)
        with db.cursor() as cursor:
            cursor.execute(sql)
    db.commit()

# 데이터 Input

## 1. COMtbl 입력
* column : C_code(종목코드), C_name(회사명)
* PK : C_code

In [14]:
df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

name_list = df['회사명'].tolist()
code_list = df['종목코드'].tolist()
code_list = list(map(str,code_list))
code_list = [x.zfill(6) for x in code_list]

code_dict = { code : value for code, value in zip(code_list,name_list)} # code와 회사 이름 딕셔너리

exist_num = pd.read_sql("select count(*) from comtbl",db).values[0][0]

# COMtbl에 회사 이름과 코드 집어넣기
sql = """
    insert ignore into comtbl values (%s, %s)
"""

with db.cursor() as cursor:
    cursor.executemany(sql,list(code_dict.items()))
    db.commit()

insert_num = pd.read_sql("select count(*) from comtbl",db).values[0][0]

print(f'작업이 완료되었습니다. {insert_num - exist_num}개의 데이터가 업데이트 되었습니다.')

#결과 확인
sql = """
    select * from comtbl
"""
result = pd.read_sql(sql,db)

작업이 완료되었습니다. 0개의 데이터가 업데이트 되었습니다.


## 2. Ftbl 입력
* column : C_code(종목코드), dateS(시작일), dateE(종료일), BPS, EPS, Issued_Shares
* Index : C_code(Foreign Key), dateS, dateE

In [ ]:
code_list = pd.read_sql("select distinct C_code From comtbl",db)['C_code'].tolist()

for code in code_list:
    print(code)
    fs_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701' %code
    fs_page = requests.get(fs_url)
    soup = BeautifulSoup(fs_page.text, "html.parser")
    #수정평균주식수
    stock = soup.find_all('tr', class_= 'c_grid1_1 rwf acd_dep2_sub')
    CR_Data = [x.text for x in stock if '주식수' in x.text]#크롤링 데이터
    index = str(CR_Data).find('수정')
    CR_Data = str(CR_Data)[index:]
    CR_Data = CR_Data.replace('\\n', ' ')
    CR_Data = CR_Data.replace('\\xa0', '0')
    CR_Data = CR_Data.replace(',', '')
    CR_Data = CR_Data.split(' ')[:6]
    #날짜, EPS, BPS
    data_df = pd.read_html(fs_page.text, index_col = 0)[1]
    acc_month = data_df.columns.tolist()[-2].split('/')[1]
    df = []
    for year in range(2018,2021):
        dateE = dt.datetime(year,int(acc_month),1) + relativedelta(months=17) + relativedelta(days=-1)
        dateS = dt.datetime(year,int(acc_month),1) + relativedelta(months=5)
        text_list = []
        text_list.append("'" + code + "'")
        column = [x for x in data_df.columns.tolist() if f'{year}/{acc_month}' in x]
        index = [x for x in data_df.index.tolist() if 'BPS계산' in x][0]
        bps = data_df.loc[index][column].values[0]
        text_list.append(bps)
        index = [x for x in data_df.index.tolist() if 'EPS계산' in x][0]
        eps = data_df.loc[index][column].values[0]
        text_list.append(eps)
        if np.nan not in text_list:
            text_list.insert(1,dateE)
            text_list.insert(1,dateS)
            index = data_df.columns.tolist().index(f'{year}/{acc_month}')
            text_list.append(CR_Data[index])
            df.append(text_list)
        else:
            print('None값 존재')
    df = pd.DataFrame(df)
    
#insert_data(db,df,'Ftbl')
#print(str(num) + ' ' + code + ' 데이터 삽입완료')

000020
000040
000050
000060
000070
000080
000100
000120
000140
000150
000180
000210
000220
000230
000240
000250
000270
000300
000320
000370
000390
000400
000430
000440
000480
000490
000500
000520
000540
000590
000640
000650
000660
000670
000680
000700
000720
000760
000810
000850
000860
000880
000890
000910
000950
000970
000990
001000
001020
001040
001060
001070
001080
001120
001130
001140
001200
001210
001230
001250
001260
001270
001290
001340
001360
001380
001390
001420
001430
001440
001450
001460
001470
001500
001510
001520
001530
001540
001550
001560
001570
001620
001630
001680
001720
001740
001750
001770
001780
001790
001800
001810
001820
001840
001880
001940
002020
002030
002070
002100
002140
002150
002170
002200
002210
002220
002230
002240
002270
002290
002310
002320
002350
002360
002380
002390
002410
002420
002450
002460
002600
002620
002630
002680
002690
002700
002710
002720
002760
002780
002790
002800
002810
002820
002840
002870
002880
002900
002920
002960
002990
003000
003010